In [2]:
"""Specification Test:  Non-nested hypotheses: Model 2.

Swiss Metro Case Study
Demand Modeling: 1.202
Spring 2019
Alexandra Berke

"""

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio

pandas = pd.read_table("../Data/swissmetro.dat")
database = db.Database("swissmetro",pandas)

from headers import *

# Specify parameters to estimate
#1: the name of the parameter,
#2: the default value,
#3: a lower bound (or None, if no bound is specified),
#4: an upper bound, (or None, if no bound is specified),
#5: a flag that indicates if the parameter must be estimated (0) or if it keeps its default value.
ASC_CAR	 = Beta('ASC_CAR',0,-20,20,0)
ASC_SBB	 = Beta('ASC_SBB',0,-10,10,1)
ASC_SM	 = Beta('ASC_SM',0,-20,20,0)
B_CAR_COST	 = Beta('B_CAR_COST',0,-10,10,0)
B_HE	 = Beta('B_HE',0,-10,10,0)
B_SM_COST	 = Beta('B_SM_COST',0,-10,10,0)
B_CAR_TIME	 = Beta('B_CAR_TIME',0,-10,10,0)
B_TRAIN_TIME	 = Beta('B_TRAIN_TIME',0,-10,10,0)
B_SM_TIME	 = Beta('B_SM_TIME',0,-10,10,0)
B_TRAIN_COST	 = Beta('B_TRAIN_COST',0,-10,10,0)
B_SENIOR	 = Beta('B_SENIOR',0,-10,10,0)
B_GA	 = Beta('B_GA',0,-10,80,0)
B_FIRST	 = Beta('B_FIRST',0,-10,80,0)
B_SM_MALE = Beta('B_SM_MALE',0,-10,10,0)
B_CAR_MALE = Beta('B_CAR_MALE',0,-10,10,0)
B_TRAIN_MALE = Beta('B_TRAIN_MALE',0,-10,10,0)


# Define here arithmetic expressions for name that are not directly 
# available from the data

SENIOR  = DefineVariable('SENIOR', AGE   ==  5,database )
LOW_INCOME  = DefineVariable('LOW_INCOME', INCOME  ==  1,database )
HIGH_INCOME  = DefineVariable('HIGH_INCOME', INCOME  ==  3,database )
CAR_AV_SP  = DefineVariable('CAR_AV_SP', CAR_AV    *  (  SP   !=  0  ),database)
SM_COST  = DefineVariable('SM_COST', SM_CO   * (  GA   ==  0  ),database)
TRAIN_AV_SP  = DefineVariable('TRAIN_AV_SP', TRAIN_AV    *  (  SP   !=  0  ),database)
TRAIN_COST  = DefineVariable('TRAIN_COST', TRAIN_CO   * (  GA   ==  0  ),database)
one  = DefineVariable('one',1,database)

#Utilities
Car_SP = ASC_CAR * one + B_CAR_TIME * CAR_TT + B_CAR_COST * CAR_CO + B_SENIOR * SENIOR \
        + B_CAR_MALE * MALE
SBB_SP = ASC_SBB * one + B_TRAIN_TIME * TRAIN_TT + B_TRAIN_COST * TRAIN_COST + B_HE * TRAIN_HE \
        + B_GA * GA + B_FIRST * FIRST \
        + B_TRAIN_MALE * MALE
SM_SP = ASC_SM * one + B_SM_TIME * SM_TT + B_SM_COST * SM_COST + B_HE * SM_HE + B_GA * GA \
        + B_SENIOR * SENIOR + B_FIRST * FIRST \
        + B_SM_MALE * MALE

V = {3: Car_SP,1: SBB_SP,2: SM_SP}
av = {3: CAR_AV_SP,1: TRAIN_AV_SP,2: SM_AV}

# Here we use the "biogeme" way for backward compatibility
exclude = (((  PURPOSE   !=  1  ) * (  PURPOSE   !=  3  )) + (  CHOICE   ==  0  ) + (  AGE == 6  ))>0
database.remove(exclude)

# Estimation of the model
logprob = bioLogLogit(V,av,CHOICE)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "SpecTest_SM_M2"
results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

ASC_CAR=	-1.07
ASC_SM=	-0.437
B_CAR_COST=	-0.00806
B_CAR_MALE=	0.609
B_CAR_TIME=	-0.0131
B_FIRST=	0.339
B_GA=	0.608
B_HE=	-0.0063
B_SENIOR=	-1.8
B_SM_COST=	-0.0117
B_SM_MALE=	0.187
B_SM_TIME=	-0.0111
B_TRAIN_COST=	-0.028
B_TRAIN_MALE=	-0.796
B_TRAIN_TIME=	-0.00863
                 Value   Std err     t-test       p-value  Rob. Std err  \
ASC_CAR      -1.073661  0.170517  -6.296517  3.044083e-10      0.181722   
ASC_SM       -0.437150  0.129413  -3.377938  7.303161e-04      0.130826   
B_CAR_COST   -0.008061  0.001081  -7.456763  8.859580e-14      0.001508   
B_CAR_MALE    0.608850  0.068094   8.941303  0.000000e+00      0.071382   
B_CAR_TIME   -0.013115  0.000816 -16.071651  0.000000e+00      0.001658   
B_FIRST       0.338800  0.074641   4.539059  5.650576e-06      0.079340   
B_GA          0.607558  0.188704   3.219645  1.283496e-03      0.202097   
B_HE         -0.006298  0.001067  -5.901340  3.605612e-09      0.001075   
B_SENIOR     -1.796058  0.119349 -15.048828  0.000000e+00   